In [3]:
from torchvision import models
from torchinfo import summary


model = models.alexnet(weights="AlexNet_Weights.IMAGENET1K_V1")
summary(model, (1, 3, 224, 224), device="cpu")

Layer (type:depth-idx)                   Output Shape              Param #
AlexNet                                  [1, 1000]                 --
├─Sequential: 1-1                        [1, 256, 6, 6]            --
│    └─Conv2d: 2-1                       [1, 64, 55, 55]           23,296
│    └─ReLU: 2-2                         [1, 64, 55, 55]           --
│    └─MaxPool2d: 2-3                    [1, 64, 27, 27]           --
│    └─Conv2d: 2-4                       [1, 192, 27, 27]          307,392
│    └─ReLU: 2-5                         [1, 192, 27, 27]          --
│    └─MaxPool2d: 2-6                    [1, 192, 13, 13]          --
│    └─Conv2d: 2-7                       [1, 384, 13, 13]          663,936
│    └─ReLU: 2-8                         [1, 384, 13, 13]          --
│    └─Conv2d: 2-9                       [1, 256, 13, 13]          884,992
│    └─ReLU: 2-10                        [1, 256, 13, 13]          --
│    └─Conv2d: 2-11                      [1, 256, 13, 13]         

In [4]:
with open("../datasets/imagenet_classes.txt", "r") as file:
    classes = file.read().splitlines()
    
print(f"클래스 개수 : {len(classes)}")
print(f"첫 번째 클래스 레이블 : {classes[0]}")

클래스 개수 : 1000
첫 번째 클래스 레이블 : tench


In [7]:
import torch
from PIL import Image
from torchvision import models, transforms


transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
    ]
)

device = "mps" if torch.backends.mps.is_available() else "cpu"
model = models.alexnet(weights="AlexNet_Weights.IMAGENET1K_V1").eval().to(device)

model = model.to(device)      

tensors = []
files = ["../datasets/images/airplane.jpg", "../datasets/images/bus.jpg"]
for file in files:
    image = Image.open(file)
    tensors.append(transform(image))

tensors = torch.stack(tensors)
print(f"입력 텐서의 크기 : {tensors.shape}")

입력 텐서의 크기 : torch.Size([2, 3, 224, 224])


In [8]:
import torch
from torch.nn.functional import softmax
from rich.console import Console
from rich.table import Table

console = Console()

with torch.no_grad():
    outputs = model(tensors.to("mps"))  
    probs = softmax(outputs, dim=-1)
    top_probs, top_idxs = probs.topk(5)

top_classes = [[classes[idx] for idx in indices] for indices in top_idxs.tolist()]

for idx, (file, cls, prob) in enumerate(zip(files, top_classes, top_probs.tolist())):
    table = Table(title=f"{file} 추론 결과")
    table.add_column("Class", justify="left")
    table.add_column("Probability (%)", justify="right")
    for c, p in zip(cls, prob):
        table.add_row(c, f"{p * 100:.2f}%")
    console.print(table)


  ../datasets/images/airplane.jpg  
             추론 결과             
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Class         ┃ Probability (%) ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ airliner      │          66.83% │
│ warplane      │          20.12% │
│ wing          │           9.29% │
│ space shuttle │           2.89% │
│ missile       │           0.38% │
└───────────────┴─────────────────┘

   ../datasets/images/bus.jpg 추론 결과   
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Class                ┃ Probability (%) ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ streetcar            │          60.25% │
│ trolleybus           │          37.99% │
│ minibus              │           1.54% │
│ passenger car        │           0.17% │
│ recreational vehicle │           0.03% │
└──────────────────────┴─────────────────┘